In [ ]:
from openai import AzureOpenAI

In [ ]:
client = AzureOpenAI(
    api_key="generated-genki-key",  ## KEY FROM GENKI
    api_version="2024-12-01-preview",
    azure_endpoint="https://ca-gateway-genki-bot-dev.whitehill-48675d4c.swedencentral.azurecontainerapps.io",
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Hello, how are you?"}],
)

print(response.choices[0].message.content)
